In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
df = pd.read_excel('drive/MyDrive/School/3_Robo/HW1/VOL/assignment_VOL_data.xlsx')

In [37]:
df['daily % chg'] = df['Close'].pct_change()

In [38]:
df['20day vol'] = df['daily % chg'].rolling(window=20).std()

In [31]:
df['previous 20d return'] = df['Close'].pct_change(periods=20)
df['next 20 day return'] = df['Close'].shift(-20) / df['Close'] -1

In [52]:
window = 250
df['zvol20'] = (df['20day vol'] - df['20day vol'].rolling(window=window).mean()) / df['20day vol'].rolling(window=window).std()
df['zret20'] = (df['previous 20d return'] - df['previous 20d return'].rolling(window=window).mean()) / df['previous 20d return'].rolling(window=window).std()
df['zfret20'] = (df['next 20 day return'] - df['next 20 day return'].rolling(window=window).mean()) / df['next 20 day return'].rolling(window=window).std()
# print(df[['20day vol', 'previous 20d return', 'next 20 day return', 'zvol20', 'zret20', 'zfret20']].head(300))

In [42]:
for column in ['zvol20', 'zret20', 'zfret20']:
    non_nan_count = df[column].notna().sum()
    print(f"{column} non-NaN count: {non_nan_count}")


zvol20 non-NaN count: 10054
zret20 non-NaN count: 10054
zfret20 non-NaN count: 10054


In [53]:
df_clean = df.dropna(subset=['zvol20', 'zret20', 'zfret20'])
# print(df_clean)


In [ ]:
# Check for sufficient non-NaN data before proceeding
if df[['zvol20', 'zret20', 'zfret20']].notna().sum().min() > 0:
    # Proceed with analysis if there are non-NaN values
    df_clean = df.dropna(subset=['zvol20', 'zret20', 'zfret20'])

    # Ensure df_clean is not empty
    if not df_clean.empty:
        # Step 6: Bucket zvol20 into Quintiles and handle data accordingly
        df_clean['zvol20_quintile'] = pd.qcut(df_clean['zvol20'], 5, labels=False)

        # Calculate averages for plotting
        quintile_means_zret20 = df_clean.groupby('zvol20_quintile')['zret20'].mean()
        quintile_means_zfret20 = df_clean.groupby('zvol20_quintile')['zfret20'].mean()

        # Plotting
        # zvol20 vs. zret20
        quintile_means_zret20.plot()
        plt.xlabel('Quintile of zvol20')
        plt.ylabel('Average zret20')
        plt.title('Relationship between zvol20 and zret20')
        plt.show()

        # zvol20 vs. zfret20
        quintile_means_zfret20.plot()
        plt.xlabel('Quintile of zvol20')
        plt.ylabel('Average zfret20')
        plt.title('Lead-Lag Relationship between zvol20 and zfret20')
        plt.show()
    else:
        print("Dataframe 'df_clean' is empty after dropping NaN values.")
else:
    print("Insufficient non-NaN data to proceed with analysis.")

In [60]:
df_sorted = df.sort_values(by='zvol20', ascending=True)
df_sorted['group'] = pd.qcut(df_sorted['zvol20'], 5, labels=[1, 2, 3, 4, 5])
# print(df_sorted['group'].value_counts())
# Map the mean values of zvol20, zret20, and zfret20 back to the original DataFrame
# df_sorted['mean_zvol20'] = df_sorted['group'].map(group_averages['zvol20'])
# df_sorted['mean_zret20'] = df_sorted['group'].map(group_averages['zret20'])
# df_sorted['mean_zfret20'] = df_sorted['group'].map(group_averages['zfret20'])
# print(df_sorted[['group', 'mean_zvol20', 'mean_zret20', 'mean_zfret20']].head())

In [61]:
group_averages = df_sorted.groupby('group').agg({'zvol20': 'mean', 'zret20': 'mean', 'zfret20': 'mean'})
# print(group_averages)

In [62]:
df_sorted.to_excel('drive/MyDrive/School/3_Robo/HW1/VOL/modified_data.xlsx',index=False)